# packages used

In [ ]:
import os
import re
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib.cm import get_cmap
from io import StringIO
from pathlib import Path
%config InlineBackend.figure_format='retina'
plt.rcParams.update({'font.size': 18})
plt.rcParams['text.usetex'] = True #tex rendering
pathf="/.../"#replace ... with path of binary output(see output_file_description.dat for more info)
pathi = "./fgs/1lay/case3.6/"#-----------------salvatagiioooo
plt.savefig(pathi+f'1l_crck_ovrly_case3.6.pdf',dpi=300)

# bibliography - info

Author: Francesco De Rose - Cosenza (CS), 03/08/2025. This notebook has been used to make the figures shown in the "Path of Excellence" presentation titled: "Magma intrusion in the crust: implications to the crustal dynamics" with supervisors Prof. Mario La Rocca[1], Prof.ssa Eleonora Rivalta[2], Dott. Francesco Maccaferri[3].
[1]University of Calabria, Physics department, Rende (CS);
[3]Istituto Nazionale di Geofisica e Vulcanologia, Sezione di Napoli - Osservatorio Vesuviano, Napoli, Italy;
[2]Department of Physics, Section of Geophysics, University of Bologna, V.le B. Pichat 8, Bologna. 

The code version used has been the one reachable from [this link](https://zenodo.org/records/7118734), the articles which have been consulted in order to make the runs have been: 
1. "Modeling the shape and velocity of magmatic intrusions, a new numerical approach", Furst, Maccaferri, Pinel, 2023 https://doi.org/10.1029/2022JB025697
2. "Buoyancy-Driven Magma Fracture' A Mechanism for Ascent Through the Lithosphere and the Emplacement of Diamonds, Spence&Turcotte 1990 https://doi.org/10.1029/JB095iB04p05133
3. "Rivalta_etal - magma_ascent encyclopedia_volcanoes" 
4. "A review of mechanical models of dike propagation: Schools of thought, results and future directions", E. Rivalta , B. Taisne , A.P. Bunger , R.F. Katz, 2015 https://doi.org/10.1016/j.tecto.2014.10.003
5. "A quantitative study of the mechanisms governing dike propagation, dike arrest and sill formation", F. Maccaferri , M. Bonafede , E. Rivalta, 2011 https://doi.org/10.1016/j.jvolgeores.2011.09.001
6. "Modeling Dike Propagation in Both Vertical Length and Horizontal Breadth", Stephen Pansino, Adel Emadzadeh, and Benoit Taisne, 2022 https://doi.org/10.1029/2022JB024593
7. "Dynamics of magmatic intrusion: what can we learn from the comparison of analog and numerical models?", Séverine Furst, Virginie Pinel and Francesco Maccaferri, 2024 https://doi:10.30909/vol.07.01.6787
8. "Jaeger - Fundamentals of rock mechanics (2007) " https://www.researchgate.net/publication/220010592_Fundamental_of_Rock_Mechanics

# Single layer

## crack shapes evlt

In [ ]:
def plot_crack_shapes_subplots(path=".", base_filename="crack_shape_r01_i", iterations=None, scale=1.0, vert_exag=5):
    """
    Plots multiple crack shape files in subplots as filled polygons (drop shapes), with fill color 
    mapped to a colorbar based on file index.
    Parameters:
        path (str): Directory containing the .dat files
        base_filename (str): Base name before the index (default: 'crack_shape_r01_i')
        indices (list): List of iteration indices to plot (default: [9, ..., 100])
        scale (float): Scale factor for exaggerating x, z
        vert_exag (float): Vertical exaggeration factor to stretch z-axis
    """
    if iterations is None:
        iterations = [9, 19, 29, 39, 49, 59, 69, 79, 89, 99]
    ncols = 5
    nrows = 2
    cmap = cm.afmhot
    norm = mcolors.Normalize(vmin=min(iterations), vmax=max(iterations))
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 8), constrained_layout=True)
    axs = axs.flatten()
    for i, iter_num in enumerate(iterations):
        filename = os.path.join(path, f"{base_filename}{iter_num:05d}.dat")
        ax = axs[i]
        try:
            data = np.loadtxt(filename)
            x = data[:, 0] * scale
            z = data[:, 1] * scale * vert_exag
            color = cmap(norm(iter_num))
            ax.fill(x, z, color=color, alpha=0.6, label=f'{iter_num:05d}')
            ax.plot(x, z, c='gray',ls='-', linewidth=2)
            ax.plot([x[-1], x[0]], [z[-1], z[0]], 'k-', linewidth=2)
            ax.plot(x, z, 'ko', markersize=3)
            ax.set_title(f"{os.path.basename(filename)}", fontsize=9)
            ax.invert_yaxis()
            ax.set_aspect('auto')
            ax.grid(True,linestyle='dotted')
            ax.set_xlabel("x [km]",weight='bold')
            ax.set_ylabel("z [km] (exag.)",weight='bold')
        except FileNotFoundError:
            ax.set_title(f"File not found:\n{filename}", color='red')
            ax.axis('off')
    for j in range(i+1, len(axs)):
        axs[j].axis('off')
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=axs, orientation='vertical', fraction=0.05, pad=0.001)
    cbar.set_label("Iteration index")
    plt.suptitle(f"Crack shape evolution (vertical exaggeration: {vert_exag}, scale: {scale})", fontsize=21,weight='bold')

In [ ]:
plot_crack_shapes_subplots(path=pathf+"1lay-case3.6/output/", base_filename="crack_shape_r01_i", 
iterations=None, scale=2.5, vert_exag=2.0)#-----modifica caso single layer scl_factor=1
pathi = "./fgs/1lay/case3.6/"#-----------------salvatagiioooo
plt.savefig(pathi+f'1l_crck_shp_case3.6.pdf',dpi=300)

In [ ]:
def plot_crack_shapes_overlay(path=".", base_filename="crack_shape_r01_i", iterations=None, scale=1.0, vert_exag=5):
    """
    Overlays multiple crack shapes in a single plot using filled polygons.
    Parameters:
        path (str): Directory containing the .dat files
        base_filename (str): Base name before the index (e.g., 'crack_shape_r01_i')
        iterations (list): List of iteration indices to plot
        scale (float): Horizontal scaling
        vert_exag (float): Vertical exaggeration factor
    """
    if iterations is None:
        iterations = [9, 19, 29, 39, 49, 59, 69, 79, 89, 99]#modificare in base ai file delle run
    cmap = cm.afmhot#colorbar guarda galleria
    norm = mcolors.Normalize(vmin=min(iterations), vmax=max(iterations))
    fig = plt.figure(figsize=(12, 8))
    for iter_num in iterations:
        filename = os.path.join(path, f"{base_filename}{iter_num:05d}.dat")
        try:
            data = np.loadtxt(filename)
            x = data[:, 0] * scale
            z = data[:, 1] * scale * vert_exag
            color = cmap(norm(iter_num))
            plt.fill(x, z, color=color, alpha=0.4, label=f'Iter {iter_num}')
            plt.plot(x, z, 'k-', linewidth=1)
        except FileNotFoundError:
            print(f"File not found: {filename}")
            continue
    ax = plt.gca()
    ax.invert_yaxis()
    ax.set_aspect('auto')
    ax.set_xlabel("x [km]", weight='bold')
    ax.set_ylabel("z [km] (exag.)", weight='bold')
    ax.set_title("Crack shapes evolution", fontsize=21, weight='bold')
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax, orientation='vertical', fraction=0.05, pad=0.01)    
    cbar.set_label("Iteration index")
    plt.tight_layout()

In [ ]:
plot_crack_shapes_overlay(path=pathf+"1lay-case3./output/", base_filename="crack_shape_r01_i",
iterations=None, scale=2.5, vert_exag=1.0)#-----modifica caso
pathi = "./fgs/1lay/case3.6/"#-----------------salvatagiioooo
plt.savefig(pathi+f'1l_crck_ovrly_case3.6.pdf',dpi=300)

## overpressure visc evolt

In [ ]:
def plot_overpressure_single_plot(path=".", base_prefix="Overpressure_", iterations=None, dyke_num=1):
    """
    Plots overpressure and viscous pressure drop from multiple Overpressure_XXX_XX.dat files in a single plot.
    Parameters:
        path (str): Directory containing the .dat files
        base_prefix (str): Base filename before iteration and dyke number (default: 'Overpressure_')
        iterations (list): List of iteration numbers (e.g., [1, 2, 3])
        dyke_num (int): Dyke number XX
    """
    if iterations is None:
        iterations = list(range(1, 11))  # 1 to 10
    fig, ax = plt.subplots(figsize=(10, 6))
    cmap = plt.get_cmap('PuOr', len(iterations))
    for i, iter_num in enumerate(iterations):
        filename = os.path.join(path, f"{base_prefix}{iter_num:03d}_{dyke_num:02d}.dat")
        try:
            data = np.loadtxt(filename)
            element = data[:, 0]
            overp = data[:, 1]
            dpvisc = data[:, 2]
            color = cmap(i)
            label_base = f"Iter {iter_num}"
            ax.plot(element, overp, '-', color=color, label=f'{label_base} - Overp.')
            ax.plot(element, dpvisc, '--', color=color, label=f'{label_base} - Visc.')

        except FileNotFoundError:
            print(f"[WARNING] File not found: {filename}")
            continue
    ax.set_title("Overpressure and viscous pressure drop", fontsize=21, weight='bold')
    ax.set_xlabel("Element index", weight='bold')
    ax.set_ylabel("Pressure [MPa]", weight='bold')
    norm = mcolors.Normalize(vmin=min(iterations), vmax=max(iterations))
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax, orientation='vertical', fraction=0.05, pad=0.01)    
    cbar.set_label("Iteration index")
    ax.grid(True,linestyle='dotted')
    from matplotlib.lines import Line2D
    custom_lines = [Line2D([0], [0], color='black', linestyle='-', label='Overpressure'),
        Line2D([0], [0], color='black', linestyle='--', label='Viscous pressure drop')]
    ax.legend(handles=custom_lines, fontsize='large', ncol=1, loc='best')
    plt.tight_layout()

In [ ]:
plot_overpressure_single_plot(pathf+"1lay-case4/output/",
base_prefix="Overpressure_", iterations=range(1,100,10), dyke_num=1)#-----modifica caso
pathi = "./fgs/1lay/case5/"#-----------------salvatagiioooo
plt.savefig(pathi+f'ovep_vscp_case5.pdf',dpi=300)

## dyke opens evolt

In [ ]:
def plot_opening_all_in_one(path=".", base_prefix="Ut_", iterations=None, title=None, dyke_num=1):
    """
    Plots all dislocation openings Ut from multiple iterations in a single figure.
    Parameters:
        path (str): Directory containing the .dat files.
        base_prefix (str): Prefix before iteration and dyke number (default: 'Ut_').
        iterations (list): List of iteration numbers (default: 1 to 10).
        dyke_num (int): Dyke number (default: 1).
    """         
    if iterations is None:
        iterations = list(range(1, 11))
    fig, ax = plt.subplots(figsize=(10, 6))
    cmap = plt.get_cmap('berlin', len(iterations))
    for i, iter_num in enumerate(iterations):
        filename = os.path.join(path, f"{base_prefix}{iter_num:03d}_{dyke_num:02d}.dat")
        try:
            data = np.loadtxt(filename)
            element = data[:, 0]
            ut = data[:, 1]
            color=cmap(i)
            ax.plot(element, ut, color=color, label=f'Iter {iter_num:03d}')
        except FileNotFoundError:
            print(f"Warning: file not found: {filename}")
            continue
        except Exception as e:
            print(f"Error reading {filename}: {e}")
            continue
    ax.set_xlabel("Element index",weight='bold')
    ax.set_ylabel("Opening [m]",weight='bold')
    ax.set_title(title or f'Dyke opening evolution: {os.path.basename(file_path)}', weight='bold')
    ax.grid(True,linestyle='dotted')
    norm = mcolors.Normalize(vmin=min(iterations), vmax=max(iterations))
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax, orientation='vertical', fraction=0.05, pad=0.01)    
    cbar.set_label("Iteration index")
    plt.tight_layout()

In [ ]:
plot_opening_all_in_one(pathf+"1lay-test/output/",
iterations=range(1, 100,5), dyke_num=1, title="1-layer dyke opening evolution – SIM A8");#-----modifica caso
pathi = "./fgs/1lay/case8/"#-----------------salvatagiioooo
plt.savefig(pathi+f'1l_open_case8.pdf',dpi=300)

## tip veloc - max opening

In [ ]:
def plot_tip_velocity_maxopening(path=".", dyke_num=1, filename_prefix="XtZtvel_"):
    """
    Plots crack tip position, velocity, and maximum opening over iterations from XtZtvel_XX.dat.
    Parameters:
        path (str): Directory containing the file
        dyke_num (int): Dyke number XX
        filename_prefix (str): File prefix (default: 'XtZtvel_')
    """
    filename = os.path.join(path, f"{filename_prefix}{dyke_num:02d}.dat")
    try:
        try:
            data = np.loadtxt(filename, comments="#", skiprows=1)
        except ValueError:
            with open(filename) as f:
                lines = [line for line in f if len(line.strip().split()) == 4]
            data = np.array([[float(x) for x in line.strip().split()] for line in lines])
        x = data[:, 0]
        z = data[:, 1]
        v = data[:, 2]
        max_ut = data[:, 3]
        iterations = np.arange(1, len(x) + 1)
        fig, axs = plt.subplots(3, 1, figsize=(10, 10), constrained_layout=True)
        axs[0].plot(iterations, x, 'b-', label='Tip x')
        axs[0].plot(iterations, z, 'g-', label='Tip z')
        axs[0].set_ylabel("Position [km]",weight='bold')
        axs[0].set_title("Crack tip position")
        axs[0].legend()
        axs[0].grid(True,linestyle='dotted')
        axs[1].plot(iterations, v, 'r-', label='Velocity')
        axs[1].set_ylabel("Velocity [m/s]",weight='bold')
        axs[1].set_title("Crack propagation velocity")
        axs[1].grid(True,linestyle='dotted')
        axs[2].plot(iterations, max_ut, 'k-', label='max(Ut)')
        axs[2].set_xlabel("Iteration",weight='bold')
        axs[2].set_ylabel("Max opening [m]",weight='bold')
        axs[2].set_title("Maximum opening")
        axs[2].grid(True,linestyle='dotted')
        plt.suptitle(f"Crack evolution – Dyke {dyke_num:02d}", fontsize=16)
        plt.show()
    except FileNotFoundError:
        print(f"File not found: {filename}")

In [ ]:
plot_tip_velocity_maxopening(pathf+"1lay-case2/output/", dyke_num=1)
pathi = "./fgs/1lay/case1/"
plt.savefig(pathi+f'crck_shape_case7.pdf',dpi=300)

## crack tip coordinates

In [ ]:
def load_fortran_float_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    lines = [line.replace('D', 'E').replace('d', 'E') for line in lines]
    from io import StringIO
    return np.loadtxt(StringIO(''.join(lines)))
def plot_delta_parameters(path=".", dyke_num=1, filename_prefix="XtZtDELTA_"):
    """
    Plots crack tip parameters and DELTA values from XtZtDELTA_XX.dat.
    Parameters:
        path (str): Directory containing the file
        dyke_num (int): Dyke number XX
        filename_prefix (str): File prefix (default: 'XtZtDELTA_')
    """
    filename = os.path.join(path, f"{filename_prefix}{dyke_num:02d}.dat")
    try:
        data = load_fortran_float_file(filename)
        iterations = np.arange(1, data.shape[0] + 1)
        x, z = data[:, 0], data[:, 1]
        overp = data[:, 2]
        l_norm = data[:, 3]
        sig_av, sig_n, sig_max = data[:, 4], data[:, 5], data[:, 6]
        delta1, delta2, delta3 = data[:, 7], data[:, 8], data[:, 9]
        fig, axs = plt.subplots(4, 1, figsize=(12, 14), constrained_layout=True)
        axs[0].plot(iterations, x, label='x_tip', color='b')
        axs[0].plot(iterations, z, label='z_tip', color='g')
        axs[0].set_title("Crack tip coordinates")
        axs[0].set_ylabel("Position [km]",weight='bold')
        axs[0].legend()
        axs[0].grid(True,linestyle='dotted')
        axs[1].plot(iterations, overp, label='OverP_av', color='r')
        axs[1].plot(iterations, l_norm, label='L_norm', color='k')
        axs[1].set_title("Overpressure and normaliz. dyke length")
        axs[1].set_ylabel("Value")
        axs[1].legend()
        axs[1].grid(True,linestyle='dotted')
        axs[2].plot(iterations, sig_av, label=r'$\bar{\sigma_{xz}}$', color='orange')
        axs[2].plot(iterations, sig_n, label=r'${\sigma_{xz}}_n$', color='purple')
        axs[2].plot(iterations, sig_max, label=r'${\sigma_{xz}}^{max}$', color='gray')
        axs[2].set_title("Shear stress components")
        axs[2].set_ylabel("Stress [Pa]",weight='bold')
        axs[2].legend()
        axs[2].grid(True)
        axs[3].plot(iterations, delta1, label='DELTA1', color='darkred')
        axs[3].plot(iterations, delta2, label='DELTA2', color='darkgreen')
        axs[3].plot(iterations, delta3, label='DELTA3', color='darkblue')
        axs[3].set_title("DELTA values")
        axs[3].set_xlabel("Iteration",weight='bold')
        axs[3].set_ylabel("Dimensionless",weight='bold')
        axs[3].legend()
        axs[3].grid(True,linestyle='dotted')
        plt.suptitle(f"Crack evolution and DELTA values. Dyke {dyke_num:02d} - case 1", fontsize=16,weight='bold')
        plt.show()
    except FileNotFoundError:
        print(f"File not found: {filename}")
    except ValueError as e:
        print(f"Error reading file {filename}: {e}")

In [ ]:
plot_delta_parameters(pathf+"1lay-case6/output/", dyke_num=1)
pathi = "./fgs/1lay/case1/"
plt.savefig(pathi+f'crck_shape_case7.pdf',dpi=150)

## velocity vs dimensionless specncerturcotte1990

In [ ]:
def compute_theoretical_velocity(times, A0, delta_rho, g, eta):
    times_safe = np.where(times == 0, 1e-10, times)
    v_th = ((A0**2 * delta_rho * g) / (48 * eta * times_safe**2))**(1/3)
    return v_th
def load_cleaned_xtztvel_file(filename):
    from io import StringIO
    with open(filename, 'r') as f:
        lines = f.readlines()
    cleaned_lines = []
    for line in lines:
        clean_line = line.replace('D', 'E').replace('d', 'E').strip()  
        parts = clean_line.split()
        if len(parts) < 4:
            continue
        try:
            floats = list(map(float, parts[:4]))
            cleaned_lines.append(' '.join(parts[:4]))
        except ValueError:
            continue  
    if not cleaned_lines:
        raise ValueError("No valid data lines found.")
    return np.loadtxt(StringIO('\n'.join(cleaned_lines)))
def plot_xtzt_velocity(file_path, A0=1.0, g=9.81, delta_rho=300, eta=100, title=None):
    try:
        data = load_cleaned_xtztvel_file(file_path)
        x, z = data[:, 0], data[:, 1]
        v = data[:, 2]
        max_opening = data[:, 3]
        iterations = np.arange(1, len(v) + 1)
        L = np.max(z) - np.min(z)
        t = iterations  
        tstar = (A0**2 * g * delta_rho * t) / (np.pi**2 * (L / 2)**3 * eta) * 1000
        v_theoretical = compute_theoretical_velocity(t, A0, delta_rho, g, eta)
        fig, ax1 = plt.subplots(figsize=(10, 5)) 
        ax1.plot(tstar, v, label='Crack tip velocity [mm/s]', marker='o', ms=2.5,color="#2313CF")
        ax1.plot(tstar, max_opening, label='Max crack opening [m]',ms=2.8,marker='s', color="#E39C27")
        ax1.plot(tstar, v_theoretical, linestyle='--', color='red', label='v_th (SpencerTurcotte1990)')
        ax1.set_xlabel('Dimensionless time $t^*$')
        ax1.set_ylabel('Value')
        ax1.set_title(title or f'Crack velocity & opening: {os.path.basename(file_path)}', fontsize=20, weight='bold')
        ax1.legend(loc='upper right', fontsize='medium')
        ax1.grid(True, linestyle='dotted')
        plt.tight_layout()
    except Exception as e:
        print(f"Error reading file {file_path}:\n{e}")

In [ ]:
file_path = pathf+"1lay-test/output/XtZtvel_01.dat" 
plot_xtzt_velocity(file_path=file_path,A0=0.009,delta_rho=300,eta=100,g=9.81,
title="1-layer dyke propagation – SIM A8")#-----modifica caso
pathi = "./fgs/1lay/case8/"#-----------------salvatagiioooo
plt.savefig(pathi+f'1l_veln_th_case8.pdf',dpi=300)

## v vs fluid flow velocity

In [ ]:
def compute_fluid_velocity(datapath, eta, A0, crack_length_km=15.0, n_steps=100):
    """
    Cplot the fluid velocity f(s,t) from midpoint avg overpressure profiles.
    Parameters:
        datapath (Path or str): Directory containing overpressure_XXX_01.dat files.
        fixed: XtZtvel_01.dat inside the directory read in datapath.
        eta (float): Viscosity [Pa·s].
        A0 (float): Crack opening area [m^2].
        crack_length_km (float): Total crack length in km.
        n_steps (int): Number of propagation steps (default same as iteration steps 100).
    """
    datapath = Path(datapath)
    L = crack_length_km * 1e3# km _> mtrs
    s = np.linspace(0, L, n_steps)#crack length coordinates
    P_avg = []
    crack_vel = pd.read_csv(datapath / f"XtZtvel_01.dat", sep='\s+', skiprows=1, header=None)
    last = crack_vel.iloc[-1, 2]
    for i in range(1, n_steps + 1):
        file = datapath / f"Overpressure_{i:03d}_01.dat"
        data = np.loadtxt(file)
        p = data[:, 1]
        P_avg.append(np.mean(p))#average no mid-point
    P_avg = np.array(P_avg)
    P_avg = P_avg * 1e9
    dP_ds = np.gradient(P_avg, s,edge_order=2)
    f_st = -(1 / (12 * eta)) * dP_ds * A0**3#Poiseuille flow in crack
    plt.figure(figsize=(6, 4))
    plt.plot(s / 1e3, f_st, label=r"$f(s,t) = -\frac{1}{12\eta} \frac{\partial P_{\nu}}{\partial s} A_0^3$", color="navy", lw=2)
    plt.axhline(y=last, color='#C75A20', linestyle='--', label = f"$v_{{f}}$ = {last:.4f} m/s")  
    plt.xlabel("Crack length [km]",weight='bold')
    plt.ylabel("Fluid flow velocity [m/s]",weight='bold')
    plt.title("Fluid flux in laminar regime",weight='bold')
    plt.legend(loc='upper right', fontsize='small')
    plt.tight_layout()
    return s, f_st

In [ ]:
path = pathf+"1lay-test/output/" 
compute_fluid_velocity(path,eta=100,#Pa.s -----modifica caso
A0=0.009,crack_length_km=7.0);
pathi = "./fgs/1lay/case8/"#-----------------salvatagiioooo
plt.savefig(pathi+f'flux_case8.pdf', dpi=300)

## v vs depth

In [ ]:
def plot_v_vs_z(file_path, title=None):
    try:
        data = np.loadtxt(file_path, usecols=(1, 2), skiprows=1)
        z = data[:, 0] #in km
        v = data[:, 1] #in km/s
        sorted_idx = np.argsort(z)
        z = z[sorted_idx]
        v = v[sorted_idx]
        fig, ax = plt.subplots(figsize=(5, 3))  # corrected line
        ax.plot(v, z, "k-",  linewidth=2, label='Crack velocity [$m/s$]')
        ax.set_xlabel("Crack tip velocity [mm/s]",  weight='bold')
        ax.set_ylabel("Crack tip depth [km]", weight='bold')
        ax.invert_yaxis()  
        ax.grid(False)
        ax.tick_params(labelsize=10)
        ax.xaxis.set_major_locator(ticker.MaxNLocator(5))
        ax.yaxis.set_major_locator(ticker.MaxNLocator(10))
        ax.set_facecolor('white')
        ax.set_title(title or f'Crack tip velocity vs depth: {os.path.basename(file_path)}')
        ax.legend(loc='upper right', fontsize='medium')
        plt.tight_layout()
    except Exception as e:
        print(f"Error reading file {file_path}:\n{e}")

In [ ]:
file = pathf+"1lay-test/output/XtZtvel_01.dat" 
plot_v_vs_z(file, title=r"$v_i = 5.3 \, mm/s$ - SIM A8")#-----modifica caso
pathi = "./fgs/1lay/case8/"#-----------------salvatagiioooo
plt.savefig(pathi+f'1l_crckvelvdep_case8.pdf',dpi=300)

## v vs viscosity

In [ ]:
def plot_multiple_v_vs_z(files_with_viscosities, title=None):
    """
    files_with_viscosities: list of (file_path, viscosity_value) tuples
    """
    norm = mcolors.LogNorm(vmin=min(v for _, v in files_with_viscosities),
                          vmax=max(v for _, v in files_with_viscosities))
    cmap = cm.copper
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    fig, ax = plt.subplots(figsize=(5, 4))
    for file_path, viscosity in files_with_viscosities:
        try:
            data = np.loadtxt(file_path, usecols=(1, 2), skiprows=1)
            z = data[:, 0]  # depth in km
            v = data[:, 1]  # velocity in mm/s
            sorted_idx = np.argsort(z)
            z = z[sorted_idx]
            v = v[sorted_idx]
            color = cmap(norm(viscosity))
            ax.plot(v, z, '-', linewidth=2, color=color, label=f"{viscosity:.1e} Pa·s")
        except Exception as e:
            print(f"Error reading file {file_path}:\n{e}")
    ax.set_xlabel("Crack tip velocity [mm/s]", weight='bold')
    ax.set_ylabel("Crack tip depth [km]", weight='bold')
    # ax.set_xlim(-0.001, 0.58)
    # # ax.set_ylim(7.5, 10)
    ax.invert_yaxis()
    ax.tick_params(labelsize=10)
    ax.xaxis.set_major_locator(ticker.MaxNLocator(5))
    ax.yaxis.set_major_locator(ticker.MaxNLocator(10))
    ax.set_facecolor('white')
    ax.set_title(title or 'Crack tip velocity vs depth', fontsize=15, weight='bold')
    cbar = fig.colorbar(sm, ax=ax, orientation='vertical', pad=0.01)
    cbar.set_label(r'$\eta$ [Pa·s]', weight='bold')
    return fig, ax

In [ ]:
files_with_viscosities = [(pathf +"1lay-case3/output/XtZtvel_01.dat", 1e2),(pathf +"1lay-case3.1/output/XtZtvel_01.dat", 3.16e2),
    (pathf +"1lay-case3.2/output/XtZtvel_01.dat", 4e2),(pathf +"1lay-case3.3/output/XtZtvel_01.dat", 4.25e2),
(pathf +"1lay-case3.4/output/XtZtvel_01.dat", 2.5e2),
(pathf +"1lay-case3.5/output/XtZtvel_01.dat", 5.0e2),(pathf +"1lay-case3.6/output/XtZtvel_01.dat", 7.5e2),]
fig, ax = plot_multiple_v_vs_z(files_with_viscosities, title="Crack tip velocity profiles")
plt.savefig("crck_vel_prflzoomin.pdf", dpi=300)
plt.show()

## energy figures U, W_r W_f

In [ ]:
def parse_runinfo_file(filename):
    """
    get RUNINFO_01.dat from initial Etot, final U, W_r, and W_f values for each main ITERATION.
    """
    iters = []
    u_values = []
    w_r_values = []
    w_f_values = []
    etot_value = None  
    current_iter = None
    temp_u = []
    temp_w_r = []
    temp_w_f = []
    try:
        with open(filename, 'r') as f:
            for line in f:
                line = line.strip()
                if '=' in line:
                    parts = line.split('=', 1)
                    key = parts[0].strip()
                    value_str = parts[1].strip()
                    if key == 'Etot' and etot_value is None:
                        try:
                            etot_value_str = value_str.split()[0]
                            if 'NaN' not in etot_value_str:
                                etot_value = float(etot_value_str)
                        except (ValueError, IndexError):
                            pass
                    elif key == 'ITER':
                        if current_iter is not None:
                            if temp_u:
                                iters.append(current_iter)
                                u_values.append(temp_u[-1])
                                w_r_values.append(temp_w_r[-1])
                                w_f_values.append(temp_w_f[-1])
                            temp_u, temp_w_r, temp_w_f = [], [], []
                        try:
                            iter_parts = value_str.split('/')
                            current_iter = int(iter_parts[0].strip())
                        except (ValueError, IndexError):
                            current_iter = None
                    if current_iter is not None:
                        try:
                            value = float(value_str)
                            if key == 'U':
                                temp_u.append(value)
                            elif key == 'W_r':
                                temp_w_r.append(value)
                            elif key == 'W_f':
                                temp_w_f.append(value)
                        except (ValueError, IndexError):
                            continue
            if current_iter is not None and temp_u:
                iters.append(current_iter)
                u_values.append(temp_u[-1])
                w_r_values.append(temp_w_r[-1])
                w_f_values.append(temp_w_f[-1])
        print(f"Initial Etot value: {etot_value}")
        print(f"Final data lengths for U, W_r, and W_f: {len(u_values)}")
        return iters, u_values, w_r_values, w_f_values, etot_value
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return None, None, None, None, None
def plot_runinfo_data(iters, u_values, w_r_values, w_f_values, etot_value,title=None):
    plt.figure(figsize=(7, 6))
    plt.plot(iters, u_values, label='U-Gravit. energy')
    plt.plot(iters, w_r_values, label=r'$W_r -\,$ Strain energy')
    plt.plot(iters, w_f_values, label=r'$W_f -\,$ Fluid elastic energy')
    if etot_value is not None:
        plt.axhline(y=etot_value, color='r', linestyle='--', label=f'$E_t$ = {etot_value:.2f}')
    plt.xlabel('Iteration', weight='bold')
    plt.ylabel('Value', weight='bold')
    plt.title(title or f'Energy contributes w.r.t. iteration number:{os.path.basename(filename)}', fontsize=20, weight='bold')
    plt.legend(loc='best', fontsize='x-small')
    plt.tight_layout()
    pathi = "./fgs/1lay/case1/"#-----------------salvatagiioooo
    plt.savefig(pathi+f'1l_energy_case1.pdf',dpi=300)
    plt.show()
iters, u_values, w_r_values, w_f_values, etot_value = parse_runinfo_file(pathf+'/1lay-case1/output/RUNINFO_01.dat')#-----modifica caso
if iters and u_values and w_r_values and w_f_values:
    plot_runinfo_data(iters, u_values, w_r_values, w_f_values, etot_value,
    title= 'Energy contributes w.r.t. iteration number - case 1')
else:
    print("No data being plot.")

In [ ]:
def plot_deltaE(filepath, ax=None, title=""):
    iterations = []
    DeE_values = []
    with open(filepath, 'r') as f:
        for line in f:
            if "DeE" in line:
                match = re.search(r'DeE\s+\(iter=\s*(\d+)\s*\)\s*=\s*([-\d.Ee+]+)', line)
                if match:
                    iter_num = int(match.group(1))
                    DeE_val = float(match.group(2))
                    iterations.append(iter_num)
                    DeE_values.append(DeE_val)
    if ax is None:
        fig, ax = plt.subplots()
    ax.plot(iterations, DeE_values, color='black', lw=1,marker='^',ms=3, label="DeE per iteration")
    ax.set_title(title)
    ax.legend(loc='best', fontsize='x-small')
    ax.set_xlabel("Iteration",weight='bold')
    ax.set_ylabel(r"$\Delta E$ [MPa $\cdot$m]", weight='bold')
fig, axes = plt.subplots(3, 2, figsize=(10, 8), sharex=True)
axes = axes.flatten()#-----modifica caso
paths = [pathf + '/1lay-case1/output/RUNINFO_01.dat',pathf + '/1lay-case2/output/RUNINFO_01.dat',
    pathf + '/1lay-case3/output/RUNINFO_01.dat',pathf + '/1lay-case4/output/RUNINFO_01.dat',
    pathf + '/1lay-case5/output/RUNINFO_01.dat',pathf + '/1lay-case6/output/RUNINFO_01.dat']
titles = [r"SIMA1: $E_f =20 MPa \cdot m$", r"SIMA3: $E_f =14 MPa \cdot m$",
          r"SIMA3.5: $E_f =16 MPa \cdot m$",r"SIMA4: $E_f =11 MPa \cdot m$",
          r"SIMA6: $E_f =12 MPa \cdot m$", r"SIMA8: $E_f =4.6 MPa \cdot m$"]
for i in range(6):
    plot_deltaE(paths[i], ax=axes[i], title=titles[i])
plt.tight_layout()
plt.savefig(f'1l_enrgvar_case1.pdf',dpi=300)
plt.show()

## multiple dykes

In [ ]:
def load_fortran_float_file(filename):
    with open(filename, 'r') as f:
        lines = [line.replace('D', 'E').replace('d', 'E') for line in f]
    return np.loadtxt(StringIO(''.join(lines)))
def plot_deltas_multiple_dykes(path=".", dyke_nums=None, filename_prefix="XtZtDELTA_"):
    """
    Plots DELTA1, DELTA2, DELTA3 over iterations for multiple dyke numbers.

    Parameters:
        path (str): Directory containing XtZtDELTA_XX.dat files
        dyke_nums (list): List of dyke numbers (e.g., [1, 2, 3])
        filename_prefix (str): Filename prefix (default: XtZtDELTA_)
    """
    if dyke_nums is None:
        dyke_nums = [1]  #single dyke
    colors = plt.cm.tab10.colors  
    fig, axs = plt.subplots(3, 1, figsize=(12, 10), sharex=True, constrained_layout=True)
    labels = ['DELTA1', 'DELTA2', 'DELTA3']
    for i_dyke, dyke in enumerate(dyke_nums):
        filename = os.path.join(path, f"{filename_prefix}{dyke:02d}.dat")
        try:
            data = load_fortran_float_file(filename)
            iterations = np.arange(1, data.shape[0] + 1)
            delta1, delta2, delta3 = data[:, 7], data[:, 8], data[:, 9]
            axs[0].plot(iterations, delta1, label=f'Dyke {dyke:02d}', color=colors[i_dyke % len(colors)])
            axs[1].plot(iterations, delta2, label=f'Dyke {dyke:02d}', color=colors[i_dyke % len(colors)])
            axs[2].plot(iterations, delta3, label=f'Dyke {dyke:02d}', color=colors[i_dyke % len(colors)])
        except Exception as e:
            print(f"Could not read {filename}: {e}")
    for i, ax in enumerate(axs):
        ax.set_ylabel(labels[i])
        ax.grid(True)
        ax.legend()
        if i == 2:
            ax.set_xlabel("Iteration")
    plt.suptitle("DELTA parameters func. iteration for n dykes", fontsize=16)
    plt.show()

In [ ]:
path_f="1lay-case7/output/"
plot_deltas_multiple_dykes(path=path_f, dyke_nums=[1, 2, 3, 4, 5])

# Double layer

## crack shapes evlt

In [ ]:
plot_crack_shapes_subplots(pathf+"2lay-case3/output/", iterations=[0,99, 199, 299, 301], scale=2.5, vert_exag=1.0)#-----modifica caso
pathi = "./fgs/2lay/case9/"#-----------------salvatagiioooo
#plt.savefig(pathi+f'2l_crck_shp_case9.pdf',dpi=300)#double layer scl_factor=100

In [ ]:
plot_crack_shapes_subplots(pathf+"2lay-test/output/", iterations=[0,99, 199, 299, 301], scale=2.5, vert_exag=1.0)#-----modifica caso
pathi = "./fgs/2lay/case9/"#-----------------salvatagiioooo
#plt.savefig(pathi+f'2l_crck_shp_case9.pdf',dpi=300)#double layer scl_factor=100

In [ ]:
plot_crack_shapes_overlay(path=pathf+"2lay-test/output/", base_filename="crack_shape_r01_i",
iterations=[0, 99, 199,299,301], scale=2.5, vert_exag=1.0)#-----modifica caso
pathi = "./fgs/2lay/case9/"#-----------------salvatagiioooo
plt.savefig(pathi+f'2l_crck_ovrly_case9.pdf',dpi=300)

## dyke opens evolt

In [ ]:
plot_opening_all_in_one(pathf+"2lay-test/output/",
iterations=range(1, 100,5), dyke_num=1, title="2-layer dyke opening evolution – SIM B9")#-----modifica caso
pathi = "./fgs/2lay/case9/"#-----------------salvatagiioooo
plt.savefig(pathi+f'2l_open_case9.pdf',dpi=300)

## tip veloc - max opening X

In [ ]:
///

## crack tip coordinates X

In [ ]:
///

## velocity vs dimensionless specncerturcotte1990

In [ ]:
plot_xtzt_velocity(pathf+"2lay-case4/output/XtZtvel_01.dat",A0=0.009,delta_rho=500,eta=20#Pa.s -----modifica caso
,g=9.81,title="2-layer dyke propagation – SIM B4")
pathi = "./fgs/2lay/case4/"#-----------------salvatagiioooo
plt.savefig(pathi+f'2l_veln_th_case4.pdf',dpi=300)

## v vs depth

In [ ]:
plot_v_vs_z(pathf+"2lay-test/output/XtZtvel_01.dat" , title=r"$v_i = 0.8 \, mm/s$ - SIM B9")#-----modifica caso
pathi = "./fgs/2lay/case9/"#-----------------salvatagiioooo
plt.savefig(pathi+f'2l_crckvelvdep_case9.pdf',dpi=300)

## v vs viscosity

In [ ]:
files_with_viscosities = [(pathf +"2lay-case1/output/XtZtvel_01.dat", 500),(pathf +"2lay-case2/output/XtZtvel_01.dat", 20),
(pathf +"2lay-case3/output/XtZtvel_01.dat", 20),(pathf +"2lay-case4/output/XtZtvel_01.dat", 20),
(pathf +"2lay-case5/output/XtZtvel_01.dat", 95),(pathf +"2lay-case6/output/XtZtvel_01.dat", 10),(pathf +"2lay-case8/output/XtZtvel_01.dat", 5),
(pathf +"2lay-test/output/XtZtvel_01.dat", 100)]
fig, ax = plot_multiple_v_vs_z(files_with_viscosities, title="Crack tip velocity profiles - B1:B9")
plt.savefig("2lcrck_vel_prfl.pdf", dpi=300)
plt.show()

## energy tend U, W_r W_f DeE

In [ ]:
def parse_runinfo_file(filename):
    """
    get RUNINFO_01.dat file to extract initial Etot and
    the final U, W_r, and W_f values for each main ITERATION.
    """
    iters = []
    u_values = []
    w_r_values = []
    w_f_values = []
    etot_value = None  
    current_iter = None
    temp_u = []
    temp_w_r = []
    temp_w_f = []
    try:
        with open(filename, 'r') as f:
            for line in f:
                line = line.strip()
                if '=' in line:
                    parts = line.split('=', 1)
                    key = parts[0].strip()
                    value_str = parts[1].strip()
                    if key == 'Etot' and etot_value is None:
                        try:
                            etot_value_str = value_str.split()[0]
                            if 'NaN' not in etot_value_str:
                                etot_value = float(etot_value_str)
                        except (ValueError, IndexError):
                            pass
                    elif key == 'ITER':
                        if current_iter is not None:
                            if temp_u:
                                iters.append(current_iter)
                                u_values.append(temp_u[-1])
                                w_r_values.append(temp_w_r[-1])
                                w_f_values.append(temp_w_f[-1])
                            temp_u, temp_w_r, temp_w_f = [], [], []
                        try:
                            iter_parts = value_str.split('/')
                            current_iter = int(iter_parts[0].strip())
                        except (ValueError, IndexError):
                            current_iter = None
                    if current_iter is not None:
                        try:
                            value = float(value_str)
                            if key == 'U':
                                temp_u.append(value)
                            elif key == 'W_r':
                                temp_w_r.append(value)
                            elif key == 'W_f':
                                temp_w_f.append(value)
                        except (ValueError, IndexError):
                            continue
            if current_iter is not None and temp_u:
                iters.append(current_iter)
                u_values.append(temp_u[-1])
                w_r_values.append(temp_w_r[-1])
                w_f_values.append(temp_w_f[-1])
        print(f"Initial Etot value: {etot_value}")
        print(f"Final data lengths for U, W_r, and W_f: {len(u_values)}")
        return iters, u_values, w_r_values, w_f_values, etot_value
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return None, None, None, None, None
def plot_runinfo_data(iters, u_values, w_r_values, w_f_values, etot_value,title=None):
    plt.figure(figsize=(7, 6))
    plt.plot(iters, u_values, label='U-Gravit. energy')
    plt.plot(iters, w_r_values, label=r'$W_r -\,$ Strain energy')
    plt.plot(iters, w_f_values, label=r'$W_f -\,$ Fluid elastic energy')
    if etot_value is not None:
        plt.axhline(y=etot_value, color='r', linestyle='--', label=f'$E_t$ = {etot_value:.2f}')
    plt.xlabel('Iteration', weight='bold')
    plt.ylabel('Value', weight='bold')
    plt.title(title or f'Energy contributes w.r.t. iteration number:{os.path.basename(filename)}', fontsize=20, weight='bold')
    plt.legend(loc='best', fontsize='x-small')
    plt.tight_layout()
    pathi = "./fgs/2lay/case1/"#-----------------salvatagiioooo
    plt.savefig(pathi+f'2l_energy_case1.pdf',dpi=300)
iters, u_values, w_r_values, w_f_values, etot_value = parse_runinfo_file(pathf+'/2lay-case1/output/RUNINFO_01.dat')#-----modifica caso
if iters and u_values and w_r_values and w_f_values:
    plot_runinfo_data(iters, u_values, w_r_values, w_f_values, etot_value,
    title= 'Energy contributes w.r.t. iteration number - case 1')
else:
    print("No data being plot.")

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(10, 8), sharex=True)
axes = axes.flatten()#-----modifica caso
paths = [pathf + '/2lay-case3/output/RUNINFO_01.dat',pathf + '/2lay-case4/output/RUNINFO_01.dat',
    pathf + '/2lay-case5/output/RUNINFO_01.dat',pathf + '/2lay-case6/output/RUNINFO_01.dat',
    pathf + '/2lay-case1/output/RUNINFO_01.dat',pathf + '/2lay-test/output/RUNINFO_01.dat']
titles = [r"SIMB3: $\Delta K_c$ =270 MPa $\cdot \sqrt{m}$", r"SIMB4: $\Delta K_c$ =299 MPa $\cdot \sqrt{m}$",
          r"SIMB5:$\Delta K_c$ =270 MPa $\cdot \sqrt{m}$",r"SIMB6: $\Delta K_c$ =302 MPa $\cdot \sqrt{m}$",
          r"SIMB1: $\Delta K_c$ =302 MPa $\cdot \sqrt{m}$", r"SIMB9: $\Delta K_c$ =270 MPa $\cdot \sqrt{m}$"]
for i in range(6):
    plot_deltaE(paths[i], ax=axes[i], title=titles[i])
plt.tight_layout()
plt.savefig(f'2l_enrgvar_case1.pdf',dpi=300)

# eon, version history 08/09/2025

In [ ]:
!python --version
!pip show os re glob pandas numpy matplotlib 